In [12]:
import json
import sys 
sys.path.append("/home/nbuser/library")
import time
import pandas as pd
import requests 

In [13]:
'''Defining function to get population from koordinates API'''
def get_population(lat, lgt):
    url = 'https://koordinates.com/services/query/v1/vector.json'
    api_key = 'd4bb782366fb4727bbcb8d61d4524269'
    layer_id = 104612
    params = {
        'key' : api_key,
        'layer' : layer_id,
        'y' : lat,
        'x' : lgt,
        'format' : 'json'
    }
    time.sleep(0.1)
    response = requests.get(url, params=params)
    if response.status_code != 200:
        return pd.Series({'Population' : None})
    
    population = response.json()['vectorQuery']['layers']['104612']['features'][0]['properties']['C18_CNPop']
    return pd.Series({'Population' : population})

In [14]:
df = pd.read_csv("property.csv")

## Adding population column

In [15]:
df['Population'] = df.apply(lambda row: get_population(row['Latitude'], row['Longitude']), axis=1)
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,Population
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,177
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,123
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,123
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,120
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,228


## Adding Deprivation Index

In [16]:
otago_df = pd.read_excel("otago730395.xlsx")
otago_df.head()

,SA12018_code,NZDep2018,NZDep2018_Score,URPopnSA1_2018,SA22018_code,SA22018_name
0,7000000,10.0,1245.0,141,100100,North Cape
1,7000001,10.0,1245.0,114,100100,North Cape
2,7000002,NaN,NaN,0,100300,Inlets Far North District
3,7000003,10.0,1207.0,225,100100,North Cape
4,7000004,9.0,1093.0,138,100100,North Cape


In [17]:
index = otago_df[['NZDep2018','SA12018_code', 'NZDep2018_Score']]
df = df.merge(right=index, left_on='SA1', right_on='SA12018_code')
df.head()

,Bedrooms,Bathrooms,Address,Land area,CV,Latitude,Longitude,SA1,0-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60+ years,Suburbs,Population,NZDep2018,SA12018_code,NZDep2018_Score
0,5,3.0,"106 Lawrence Crescent Hill Park, Auckland",714,960000,-37.012920,174.904069,7009770,48,27,24,21,24,21,Manurewa,177,6.0,7009770,997.0
1,5,3.0,"8 Corsica Way Karaka, Auckland",564,1250000,-37.063672,174.922912,7009991,42,18,12,21,15,30,Karaka,123,1.0,7009991,881.0
2,6,4.0,"243 Harbourside Drive Karaka, Auckland",626,1250000,-37.063580,174.924044,7009991,42,18,12,21,15,30,Karaka,123,1.0,7009991,881.0
3,2,1.0,"2/30 Hardington Street Onehunga, Auckland",65,740000,-36.912996,174.787425,7007871,42,6,21,21,12,15,Onehunga,120,2.0,7007871,908.0
4,3,1.0,"59 Israel Avenue Clover Park, Auckland",601,630000,-36.979037,174.892612,7008902,93,27,33,30,21,33,Clover Park,228,9.0,7008902,1091.0


In [ ]:
df.to_csv('\property_with_population.csv', index = False)

In [ ]:
url = 'https://www.educationcounts.govt.nz/find-school?&address='
response = requests.get(url + '5%20Schopolo%20Place%20Schnapper%20Rock%20Auckland&lat=-36.76021345&lng=174.680169083')
print(response)